.. _configuring_pytest_notebook:

# Configuring pytest-notebook

Testing can be configured *via*;

1. The pytest configuration file.
2. The pytest command-line interface.
3. The notebook and cell level metadata.

To see the options available for (1) and (2), `pytest -h` can be used (look for options starting `nb`), and for (3) see :ref:`nb_metadata_schema`.

To access notebook metadata, either open the notebook as a text document, or use the Jupyter Notebook interface:

![Accessing notebook metadata.](images/nb_metadata.png)

In [2]:
import nbformat

In [7]:
%load_ext pytest_notebook.ipy_magic

## Ignoring Elements of the Notebook

In [62]:
notebook = nbformat.v4.new_notebook(
    cells=[
        nbformat.v4.new_code_cell(
            "print(1)",
            execution_count=1,
            outputs=[]),
        nbformat.v4.new_code_cell(
            "print(1)",
            execution_count=3,
            outputs=[
                nbformat.v4.new_output(
                    output_type="stream",
                    name="stdout",
                    text="2\n")])
    ])

In [63]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files

***
(nbformat.writes(notebook), "test_notebook1.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpi00vz37j
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook1.ipynb F                                                   [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook1) ____________________
pytest_notebook.nb_regression.NBRegressionError: 
--- expected
+++ obtained
## inserted before /cells/0/outputs/0:
+  output:
+    output_type: stream
+    name: stdout
+    text:
+      1

## replaced /cells/1/execution_count:
-  3
+  2

## modified /cells/1/outputs/0/text:
@@ -1 +1 @@
-2
+1

## added /metadata/language_info:
+  codemirror_mode:
+    name: ipython
+    version: 3
+  file_extension: .py
+  mimetype: text/x-python
+  nam

In [64]:
notebook.metadata["nbreg"] = {
    "diff_ignore": [
        "/cells/0/outputs/"
]}
notebook.cells[1].metadata["nbreg"] = {
    "diff_ignore": [
        "/outputs/0/text"
]}

In [80]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files

---
[pytest]
nb_diff_ignore =
    /metadata/language_info
    /cells/1/execution_count
---

***
(nbformat.writes(notebook), "test_notebook1.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmp10hoatev, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook1.ipynb .                                                   [100%]

===================== 1 passed, 1 warnings in 2.22 seconds =====================


In [60]:
notebook2 = nbformat.v4.new_notebook(
    cells=[
        nbformat.v4.new_code_cell(
            "print(1)",
            execution_count=1,
            outputs=[]),
        nbformat.v4.new_code_cell(
            "print(1)",
            execution_count=2,
            outputs=[])
    ])

In [61]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files

---
[pytest]
nb_diff_ignore =
    /metadata/language_info
    /cells/*/outputs/
---

***
(nbformat.writes(notebook2), "test_notebook2.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmp3mqowian, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook2.ipynb .                                                   [100%]

===================== 1 passed, 1 warnings in 2.27 seconds =====================


## Regex Pattern Replacement

In [66]:
notebook3 = nbformat.v4.new_notebook(
    cells=[
        nbformat.v4.new_code_cell(
            ('from datetime import date\n'
             'print(date.today())'),
            execution_count=1,
            outputs=[
                nbformat.v4.new_output(
                    output_type="stream",
                    name="stdout",
                    text="DATE-STAMP\n"
                )])])

In [67]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files

---
[pytest]
nb_diff_ignore =
    /metadata/language_info
---

***
(nbformat.writes(notebook3), "test_notebook3.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpnaw18gqe, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook3.ipynb F                                                   [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook3) ____________________
pytest_notebook.nb_regression.NBRegressionError: 
--- expected
+++ obtained
## modified /cells/0/outputs/0/text:
@@ -1 +1 @@
-DATE-STAMP
+2019-08-12


===================== 1 failed, 1 warnings in 2.12 seconds =====================


In [68]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files

---
[pytest]
nb_diff_ignore =
    /metadata/language_info
nb_diff_replace =
    /cells/*/outputs/*/text \d{2,4}-\d{1,2}-\d{1,2} "DATE-STAMP"
---

***
(nbformat.writes(notebook3), "test_notebook3.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpub9_yliq, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook3.ipynb .                                                   [100%]

===================== 1 passed, 1 warnings in 2.14 seconds =====================


## Ignoring Exceptions

To mark expected exceptions from a notebook cell, tag the cell as `raises-exception`:

![Tagging exceptions.](images/nb_tag_except.png)

In [69]:
notebook4 = nbformat.v4.new_notebook(
    cells=[
        nbformat.v4.new_code_cell(
        'raise Exception("expected error")',
            execution_count=1,
            outputs=[
                nbformat.v4.new_output(
                    output_type="error",
                    ename="Exception",
                    evalue="expected error",
                    traceback=[]
                )])])
notebook4.metadata["nbreg"] = {
    "diff_ignore": [
        "/metadata/language_info",
        "/cells/0/outputs/0/traceback"
]}

In [70]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files

***
(nbformat.writes(notebook4), "test_notebook4.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpn07iwm0v
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook4.ipynb F                                                   [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook4) ____________________
nbconvert.preprocessors.execute.CellExecutionError: An error occurred while executing the following cell:
------------------
raise Exception("expected error")
------------------

---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
<ipython-input-1-ba9385a0cebd> in <module>
----> 1 raise Exception("expected error")

Exception: expected err

In [73]:
notebook4.cells[0].metadata["tags"] = ["raises-exception"]

In [74]:
%%pytest --color=yes --disable-warnings --nb-test-files

***
(nbformat.writes(notebook4), "test_notebook4.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmp3bk9mzba
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook4.ipynb .                                                   [100%]

===================== 1 passed, 1 warnings in 2.45 seconds =====================


## Skipping Notebooks

To add the [pytest skip decorator](http://doc.pytest.org/en/latest/skipping.html#skipping-test-functions) to a notebook, you can add `skip=True` to the notebook metadata.

In [75]:
notebook5 = nbformat.v4.new_notebook()
notebook5.metadata["nbreg"] = {
    "skip": True,
    "skip_reason": "Not ready for testing."
}
notebook5

{'nbformat': 4,
 'nbformat_minor': 2,
 'metadata': {'nbreg': {'skip': True,
   'skip_reason': 'Not ready for testing.'}},
 'cells': []}

In [77]:
%%pytest -v --color=yes -rs --nb-test-files

***
(nbformat.writes(notebook5), "test_notebook5.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpmah5vrt1
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collecting ... collected 1 item

test_notebook5.ipynb::nbregression(test_notebook5) SKIPPED               [100%]

=========================== short test summary info ============================
SKIPPED [1] test_notebook5.ipynb: Not ready for testing.
========================== 1 skipped in 0.06 seconds ===========================


## Post-processors

In [95]:
notebook6 = nbformat.v4.new_notebook(
    cells=[
        nbformat.v4.new_code_cell(
            (
                "for i in range(5  ) :\n"
                "  x=i\n"
                "  a ='123'# comment\n"
                "c = ['a fairly long line of text', "
                "'another fairly long line of text',\n"
                "'yet another fairly long line of text']"
            ),
            execution_count=1,
            outputs=[])
    ])

In [96]:
%%pytest --color=yes --disable-warnings --nb-test-files --nb-force-regen

---
[pytest]
nb_diff_ignore =
    /metadata/language_info
nb_post_processors =
    coalesce_streams
    blacken_code
---

***
(nbformat.writes(notebook6), "test_notebook6.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpxc1klind, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.3.1
collected 1 item

test_notebook6.ipynb F                                                   [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook6) ____________________
pytest_notebook.nb_regression.NBRegressionError: Files differ and --nb-force-regen set, regenerating file at:
- /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpxc1klind/test_notebook6.ipynb
----------------------------- Captured stderr call -----------------------------
Diff before regeneration:

--- expected
+++ obtained
## modified /cells/0/source:
@@ -1,5 +1,8 @@
-for i in range(5  ) :
-  x=i
-  a ='1